In [ ]:
from sqlalchemy import select, func, desc
from app.models.lead import Lead
from app.models.chat_log import ChatLog

# ... inside the function
subq = (
    select(
        ChatLog.lead_id,
        func.max(ChatLog.timestamp).label("max_ts")
    )
    .group_by(ChatLog.lead_id)
    .subquery()
)

stmt = (
    select(Lead, ChatLog)
    .join(subq, Lead.id == subq.c.lead_id)
    .join(ChatLog, (ChatLog.lead_id == Lead.id) & (ChatLog.timestamp == subq.c.max_ts))
    .where(Lead.tenant_id == tenant_id)
    .order_by(desc(subq.c.max_ts))
)
